In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [2]:
#useful functions for image generation 

def abs_sobel_thresh(img, orient='x', sobel_kernel = 3, thresh_min=0, thresh_max=255):
    #convert image to grayscale 
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #apply x and y  gradient and convert to absolute values
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    #rescale to 8 bit integer 
    scaled_sobel = np.uint8(abs_sobel/(np.max(abs_sobel)/255))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output    

def hls_threshold(img,thresh_min=0, thresh_max=255):
    #convert color image in hls channels
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    #extract s channel
    s_channel = hls[:,:,2]
    #create binary image copy and apply thresholds
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh_min) & (s_channel <= thresh_max)] = 1
    
    #return result 
    return binary_output

def rgb_threshold(img,thresh_min=0, thresh_max=255):
    #extract r channel
    r_channel = img[:,:,0]
    #create binary image copy and apply thresholds
    binary_output = np.zeros_like(r_channel)
    binary_output[(r_channel > thresh_min) & (r_channel <= thresh_max)] = 1
    
    #return result 
    return binary_output

def hsv_threshold(img,thresh_min=50, thresh_max=255):
    #convert color image in hsv channels
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    #extract v channel
    v_channel = hsv[:,:,2]
    #create binary image copy and apply thresholds
    binary_output = np.zeros_like(v_channel)
    binary_output[(v_channel > thresh_min) & (v_channel <= thresh_max)] = 1 
        
    #return result 
    return binary_output

def find_lane_pixels(binary_warped):
    # Take histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the starting point for the left and right lines - these will be left and right peaks of histogram
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Define Hyperparameters
    # number of sliding windows
    nwindows = 9
    # width of the windows +/- margin
    margin = 100
    # minimum number of pixels found to recenter window
    minpix = 50

    # Calculate height of windows 
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # For loop to step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If > 50, recenter next window to new mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img
    


def fit_polynomial(binary_warped):
    # Find lane pixels
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each lane
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return out_img, left_fit, right_fit, ploty, left_fitx, right_fitx



In [3]:
import glob 
#from slide_window import slide_window
%matplotlib qt

#Camera Calibration 

#List of all images in the folder 'camera_cal' using glob API
imgs = glob.glob("C:/Users/t0794dm/Desktop/SDC_ND/project_2/CarND-Advanced-Lane-Lines/camera_cal/calibration*.jpg")

#object and image points arrays 
objpoints = [] #image points - hold 2D coordinates of the distorted test image
imgpoints = [] #object points - hold 3D coordinates of real undistorted chessboard corner points 

#object points
objpts = np.zeros((6*9,3),np.float32)
objpts[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
#objpoints

#image points 
for index, name in enumerate(imgs):
    #read the images 
    img = mpimg.imread(name)
    
    #convert the image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    #find chessboard corners in the grayscaled image 
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
    #corners

    #if 9X6 corner points exist, add those coordinates to the imgpoints array
    if ret == True: 
        imgpoints.append(corners)
        objpoints.append(objpts)

        #again, if corners exists, then draw them on the copy of the image - (corners found)
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)

        
#find dimentions of the images 
img = mpimg.imread ('C:/Users/t0794dm/Desktop/SDC_ND/project_2/CarND-Advanced-Lane-Lines/camera_cal/calibration3.jpg')
img_dim = (img.shape[1], img.shape[0])

#calibrate camera 
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_dim, None, None)

In [104]:
#image processing (single image and frame by frame for videos)
#using mtx and dist from cv2.calibrateCamera() function 
def process_image(img):
    #undistort 
    undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)        
    #create binary image 
    binary_image = np.zeros_like(undistorted_img[:,:,0])
    # Apply sobel threshold
    x_gradient = abs_sobel_thresh(undistorted_img, orient = 'x',sobel_kernel = 3, thresh_min=15, thresh_max=255) 
    y_gradient = abs_sobel_thresh(undistorted_img, orient = 'y',sobel_kernel = 3, thresh_min=25, thresh_max=255)  
    # apply color threshold 
    s_threshold = hls_threshold(undistorted_img,thresh_min=110, thresh_max=255) 
    v_threshold = hsv_threshold(undistorted_img,thresh_min=60, thresh_max=255)
    binary_image[(((x_gradient==1) & (y_gradient == 1)) | ((s_threshold==1) & (v_threshold==1)))] = 255 
  
    
    #perspective transform 
    # map 4 rectangular source points to 4 destination points 
    img_dim = (binary_image.shape[1],binary_image.shape[0]) #[1280,720]
    src = np.float32(   #change these values to represent trapezoid
        [[515,455], #[551,486] #[555,455]#[550,455]#520
         [745,455], #[765,486] #[725,455]#[720,455]#740
         [1280,675], #[1093,682] #[1280,680]#[1280,685]
         [0,675]]) #[290,690] #[0,680]#[0,685]
    dst = np.float32(
        [[0,0],#[256,0]
         [1280,0],#[1024,0]
         [1280,720], #[1024,720]
         [0,720]])#[256,720]
    #Calulate perspective transform, M
    M = cv2.getPerspectiveTransform(src, dst)
    # Inverse Perspective transform
    Minv = cv2.getPerspectiveTransform(dst, src)
    # warped image using linear interpolation
    warped_image = cv2.warpPerspective(binary_image, M, img_dim, flags=cv2.INTER_LINEAR)
    
    #lane finding
    #first frame
    out_img, left_fit, right_fit, ploty, left_fitx, right_fitx = fit_polynomial(warped_image)
    
    #subsequent frames (for video)
    nonzero = warped_image.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  
    
    #extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, warped_image.shape[0]-1, warped_image.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    
    ##visualization on the original image##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((warped_image, warped_image, warped_image))*255
    window_img = np.zeros_like(out_img)
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    
    
    #color entire lane 
    # Recast the x and y points into usable format for cv2.fillPoly()
    left_lane_pts = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    right_lane_pts = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    lane_pts = np.hstack((left_lane_pts, right_lane_pts))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([lane_pts]), (0,255, 0))

    # Draw the lane onto the warped blank image
    #cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    #cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    #all_pts = np.hstack((left_line_pts, right_line_pts))
    #cv2.fillPoly(out_img, np.int_([all_pts]), (0,255, 0))
    poly_lines = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    ###END
    
    #view polyfit lane lines on the original image
    unwarped_image = cv2.warpPerspective(window_img, Minv, img_dim, flags=cv2.INTER_LINEAR)
    road_lines = cv2.addWeighted(img,1.0,unwarped_image,1.0,0.0)
    
    #curvature and offset of the lane lines
    #assuming lane is 30meters long and 3.7meter wide
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    #polynomial in the real world
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    y_eval = np.max(ploty) #max y value corresponding to the the bottom of the image 
    #radius of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    #offset from center
    center = (left_fitx[-1]+right_fitx[-1])/2
    dist_from_center = (center-warped_image.shape[1]/2)*xm_per_pix
    offset_from ='left'
    if dist_from_center <=0:
        offset_from ='right'
  
    #print radius of curvature on the image 
    cv2.putText(road_lines,'left_ROC = '+str(round(left_curverad,3))+'m',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(road_lines,'right_ROC = '+str(round(right_curverad,3))+'m',(50,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(road_lines,'offset from center = '+str(abs(round(dist_from_center,3)))+'m to '+ offset_from+' from center',(50,150),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)

    return road_lines


    

In [105]:
#image processing
#load test images from folder 'test_images'
test_images = glob.glob("C:/Users/t0794dm/Desktop/SDC_ND/project_2/CarND-Advanced-Lane-Lines/test_images/*.jpg")
for index, name in enumerate(test_images):
    #read the images 
    img = cv2.imread(name)
    road_lines = process_image(img)
    #plt.imshow(road_lines)
    #write image in the lines_on_test_image folder 
    output_name = 'final_output_test_images'+str(index)+'.jpg'
    #change directory to save images in a folder 
    os.chdir("C:/Users/t0794dm/Desktop/SDC_ND/project_2/CarND-Advanced-Lane-Lines/output_images") 
    cv2.imwrite(output_name,road_lines)

In [106]:
os.chdir("C:/Users/t0794dm/Desktop/SDC_ND/project_2/CarND-Advanced-Lane-Lines") 

In [107]:
#vide processing
output = 'out_test_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
#clip1 = VideoFileClip("project_video.mp4").subclip(38,42)

out_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
out_clip.write_videofile(output, audio=False)

t:   0%|                                                                    | 2/1260 [00:00<01:21, 15.38it/s, now=None]

Moviepy - Building video out_test_video.mp4.
Moviepy - Writing video out_test_video.mp4



Moviepy - Done !
Moviepy - video ready out_test_video.mp4


In [108]:
HTML("""
<video  width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))